In [193]:
import pandas as pd
import numpy as np
import sys
sys.path.append("../..")
from data.dataset import DATASET as mainDf
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif

In [194]:
# write the necessary functions
# df = mainDf.loc[mainDf['risk1'].notnull()]
# t = np.where(np.isnan(x))

def getRowsForRisks(df, columns):
    partialDf = []
    for column in columns:
        data = df.loc[df[column].notnull()]
        partialDf.append(data)

    return partialDf

'''
def cleanDataframe(data):
    data = np.nan_to_num(data)
    data = data.drop(columns=['country','city'])
    return data
'''

def getImportantFeatures(df, label, risks):
    y = df.loc[:, label]
    x = df.loc[:, df.columns != label]

    #risks.remove(label)
    #print(risks)
    
    x = x.drop(columns=['country','city','country_code'])
    cols = [col for col in x.columns if col.lower()[:4] != 'risk']
    x = x[cols]

    l = x.columns[x.isna().any()].tolist()
    print(l)

    # Applying select K-best
    bestFeatures = SelectKBest(score_func=f_classif, k=10)
    fit = bestFeatures.fit(x,y)
    dfscores = pd.DataFrame(fit.scores_)
    dfcolumns = pd.DataFrame(x.columns)

    #Concat two dataframes for better visualization
    featureScores = pd.concat([dfcolumns,dfscores],axis=1)
    featureScores.columns = ['Specs','Score']
    print(featureScores.nlargest(10,'Score'))  #print 10 best features

In [187]:
display(mainDf)
print(mainDf.info)

,city,latitude,longitude,population,country_code,c40,risk0,risk1,risk2,risk3,...,SDG 6.4.1. Services Water Use Efficiency,SDG 6.4.1. Water Use Efficiency,SDG 6.4.2. Water Stress,Seasonal variability (WRI),Total internal renewable water resources per capita,Total population with access to safe drinking-water (JMP),Total renewable water resources per capita,Total water withdrawal per capita,Urban population with access to safe drinking-water (JMP),country
0,Aalborg,57.0337,9.9166,122219.0,DNK,False,NaN,NaN,NaN,NaN,...,558.335628,368.612902,20.040562,1.300000,1046.705025,100.0,1046.705025,129.285516,100.0,Denmark
1,Aarhus,56.1572,10.2107,237551.0,DNK,False,NaN,NaN,NaN,NaN,...,558.335628,368.612902,20.040562,1.300000,1046.705025,100.0,1046.705025,129.285516,100.0,Denmark
2,Copenhagen,55.6786,12.5635,1085000.0,DNK,False,NaN,2.0,NaN,2.0,...,558.335628,368.612902,20.040562,1.300000,1046.705025,100.0,1046.705025,129.285516,100.0,Denmark
3,Esbjerg,55.4670,8.4500,72205.0,DNK,False,NaN,NaN,NaN,NaN,...,558.335628,368.612902,20.040562,1.300000,1046.705025,100.0,1046.705025,129.285516,100.0,Denmark
4,Frederikshavn,57.4337,10.5333,24103.0,DNK,False,NaN,2.0,NaN,NaN,...,558.335628,368.612902,20.040562,1.300000,1046.705025,100.0,1046.705025,129.285516,100.0,Denmark
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1206,Teaoraereke,1.3333,173.0000,5105.0,KIR,False,NaN,NaN,NaN,NaN,...,110.288104,80.302831,68.505925,2.532741,-1779.609434,66.9,0.000000,414.127038,87.3,Kiribati
1207,Trindade,0.3000,6.6667,6636.0,STP,False,NaN,NaN,NaN,NaN,...,17.162254,7.489641,1.876147,2.502891,10526.874919,97.1,10526.874919,197.499626,98.9,Sao Tome and Principe
1208,Victoria,-4.6236,55.4544,26450.0,SYC,False,NaN,NaN,NaN,NaN,...,120.471023,89.323035,69.243227,1.601611,-2157.596566,95.7,0.000000,142.089651,95.7,Seychelles
1209,Warsaw,52.2167,21.0333,1790658.0,POL,True,0.0,0.0,2.0,0.0,...,153.691550,43.570222,34.890966,0.600000,1412.266377,98.3,1594.069324,265.616741,99.3,Poland


<bound method DataFrame.info of                   city  latitude  longitude  population country_code    c40  \
0              Aalborg   57.0337     9.9166    122219.0          DNK  False   
1               Aarhus   56.1572    10.2107    237551.0          DNK  False   
2           Copenhagen   55.6786    12.5635   1085000.0          DNK  False   
3              Esbjerg   55.4670     8.4500     72205.0          DNK  False   
4        Frederikshavn   57.4337    10.5333     24103.0          DNK  False   
...                ...       ...        ...         ...          ...    ...   
1206       Teaoraereke    1.3333   173.0000      5105.0          KIR  False   
1207          Trindade    0.3000     6.6667      6636.0          STP  False   
1208          Victoria   -4.6236    55.4544     26450.0          SYC  False   
1209            Warsaw   52.2167    21.0333   1790658.0          POL   True   
1210  Abasan Al-Kabira   31.3231    34.3440     32000.0          PSE  False   

      risk0  risk1 

In [189]:
a = mainDf.columns.values.tolist()
#print(a)
risks = a[6:13]
risks_index = [i for i in range(6,14)]
print(risks)
print(risks_index)

['risk0', 'risk1', 'risk2', 'risk3', 'risk4', 'risk5', 'risk6']
[6, 7, 8, 9, 10, 11, 12, 13]


In [190]:
mylist = getRowsForRisks(mainDf, risks)
print(len(mylist))
for i in range(len(mylist)):
    print("Only",mylist[i].shape[0],"rows are available for risk",i)

7
Only 87 rows are available for risk 0
Only 148 rows are available for risk 1
Only 262 rows are available for risk 2
Only 184 rows are available for risk 3
Only 98 rows are available for risk 4
Only 65 rows are available for risk 5
Only 59 rows are available for risk 6


In [191]:
#display(mylist[0])

In [195]:
for i in range(len(risks)):
    getImportantFeatures(mylist[i], risks[i], risks)

[]
                                                Specs      Score
3                                                 c40  87.682714
58  Gross domestic product (GDP), total (2017 PPP ...   4.032056
42            Population ages 65 and older (millions)   3.234697
53  Working poor at PPP$3.20 a day (% of total emp...   2.961307
28                        Inequality in education (%)   2.900850
5   Population with at least some secondary educat...   2.875645
4   Population with at least some secondary educat...   2.794997
18  Foreign direct investment, net inflows (% of GDP)   2.734514
6   Population with at least some secondary educat...   2.687594
9      Share of seats in parliament (% held by women)   2.518464
[]
                                                 Specs      Score
3                                                  c40  56.020761
140                            SDG 6.4.2. Water Stress   2.384240
132  MDG 7.5. Freshwater withdrawal as % of total r...   2.284233
16          Sex